# Object Detection Training and Testing Script

This is the script used to train and test the model used for the object detection in the BPMN Redrawer project.

Remember to change the directories paths if needed.

Mount google drive directories.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Install Detectron2

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

Import the needed packages

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

Add BoxMode = XYWH_ABS to the training annotations if needed.

In [ ]:
# from detectron2.structures import BoxMode

# def get_annotations_by_id(annotations):
#   for annot in annotations:
#     annot["bbox_mode"] = BoxMode.XYWH_ABS

#   return annotations

# with open("/content/gdrive/MyDrive/SPM21/object_detection/annotations_train.json") as jfp:
#   dataset_dicts = json.load(jfp)
#   print(get_annotations_by_id(dataset_dicts.get("annotations")))

Register the detectron datasets from the coco datasets.

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/gdrive/MyDrive/prova-OD/annotations_train.json", "/content/gdrive/MyDrive/prova-OD/train")
register_coco_instances("my_dataset_val", {}, "/content/gdrive/MyDrive/prova-OD/annotations_train.json", "/content/gdrive/MyDrive/prova-OD/train")
register_coco_instances("my_dataset_test", {}, "/content/gdrive/MyDrive/prova-OD/annotations_test.json", "/content/gdrive/MyDrive/prova-OD/test")

val = MetadataCatalog.get("my_dataset_val")
print(val)

Train the model

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = () 
# cfg.TEST.EVAL_PERIOD = 100
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 8500    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024   # faster, and good enough for this toy dataset (default: 512)
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 26  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.OUTPUT_DIR = "/content/gdrive/MyDrive/prova-OD/output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
# trainer.resume_or_load(resume=False)
trainer.resume_or_load()
trainer.train()

View train report.

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir "/content/gdrive/MyDrive/prova-OD/output"

In [ ]:
# drive.mount("/content/gdrive", force_remount=True)

Make prediction on an image and visualize it.

In [ ]:
from detectron2.utils.visualizer import ColorMode
import matplotlib.pyplot as plt
import json

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.OUTPUT_DIR = "/content/gdrive/MyDrive/prova-OD/output"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the trained model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 26
predictor = DefaultPredictor(cfg)

img = cv2.imread(f'/content/gdrive/MyDrive/prova-OD/test/img4.png')
# cv2_imshow(img)
outputs = predictor(img)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
#print(json.dumps(outputs, indent=4))
print(outputs.get("instances").get_fields())
v = Visualizer(img[:, :, ::-1],
                metadata=val,
                scale=0.8,
                instance_mode=ColorMode.IMAGE_BW
)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

Evaluate the model.

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_test", output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`